# Clasificaciones de imagenes con YOLO

##Importar los paquetes

In [ ]:
!pip install ultralytics fiftyone

In [2]:
import zipfile
import os
import shutil
import random
import fiftyone               as fo
import fiftyone.utils.random        as four
import matplotlib.pyplot          as plt
from ultralytics          import YOLO
from google.colab         import drive

Migrating database to v0.20.1


INFO:fiftyone.migrations.runner:Migrating database to v0.20.1


##BUILD ROBOCUP_2019 Dataset

###Dataset details
RoboCup Dataset contains 196195 images.

8 main parent categories that contain 180 children categories.

70% for train

20% for valid

10% for test

###Build dataset

###Connect to my oneDrive

In [3]:
drive.mount('/content/drive')
ruta_archivo_zip = '/content/drive/MyDrive/Colab Notebooks/data_subclass.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

Mounted at /content/drive


###transforming data to suit the yolo model

In [4]:
def transform_dataset(root_dir, output_dir, train_ratio=0.8):
    # Create the output directory
    os.makedirs(output_dir, exist_ok=True)
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    
    # Get the list of classes in the root directory
    classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    
    for class_name in classes:
        class_dir = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        
        # Get the list of images in the current class
        images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Shuffle the images randomly
        random.shuffle(images)
        
        # Calculate the number of training images based on the specified ratio
        train_size = int(len(images) * train_ratio)
        
        # Create the class directories in the training and validation sets
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)
        
        # Copy the training images
        for img in images[:train_size]:
            src_path = os.path.join(class_dir, img)
            dst_path = os.path.join(train_class_dir, f"{os.path.splitext(img)[0]}_{class_name}.png")
            shutil.copy(src_path, dst_path)
        
        # Copy the validation images
        for img in images[train_size:]:
            src_path = os.path.join(class_dir, img)
            dst_path = os.path.join(val_class_dir, f"{os.path.splitext(img)[0]}_{class_name}.png")
            shutil.copy(src_path, dst_path)

In [5]:
root_dir   = '/content/training_data'   # Path to the original dataset
output_dir  = '/content/dataset'      # Path for the resized dataset

transform_dataset(root_dir, output_dir, train_ratio=0.8)

##YOLO

###Train

In [ ]:
# Load a model
model = YOLO('yolov8x-cls.yaml').load('yolov8x-cls.pt')  # build from YAML and transfer weights

# Train the model
model.train(data='/content/dataset', epochs=100, imgsz=224, verbose=True)


                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   7375360  ultralytics.nn.modules.conv.Conv             [640

###Val

In [ ]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.top1   # top1 accuracy
metrics.top5   # top5 accuracy

###Export

In [ ]:
# Export the model
model.export(format='onnx')

In [ ]:
print(metrics.top1)
print(metrics.top5)